In [ ]:
# -*- coding: utf-8 -*-
"""Cópia de Lab1_Notebook.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1WHyGzpGcXJBCBtVA51lDY7L2h4XOSwjW

# Introduction: Modeling Cell Disruption in a High-Pressure Homogenizer

High-pressure homogenization (HPH) is a widely used method for cell disruption, particularly in bioprocessing, where intracellular products such as proteins, enzymes, or metabolites need to be released. During this process, cells are subjected to intense mechanical forces as they pass through a narrow orifice under high pressure, leading to cell wall rupture and product release.

To model the efficiency of cell disruption in an HPH, a kinetic equation is commonly used. The rate of product release, \( $\frac{dR}{dN}$ \), is described as a function of the applied pressure, the number of passes through the homogenizer, and the maximum achievable product concentration. The equation is as follows:

$$
\frac{dR}{dN} = k \cdot P^\alpha \cdot (R_m - R)
$$

where:

- \( $R$ \): Concentration of the released product (g/L).
- \( $N$ \): Number of passes through the homogenizer.
- \( $k$ \): Kinetic constant ( $\text{MPa}^{-\alpha}$ ), which depends on the cell type and process conditions.
- \( $P$ \): Homogenization pressure (MPa).
- \( $\alpha$ \): Pressure sensitivity coefficient, which describes how the pressure affects the disruption rate.
- \( $R_m$ \): Maximum product concentration (g/L), representing the upper limit of release.

This model assumes first-order kinetics with respect to the unreleased product ( $R_m - R$ ) and incorporates a power-law relationship between the disruption rate and pressure. The parameter \( $\alpha$ \) accounts for the nonlinear effect of pressure on cell breakage, where higher $\alpha$ values indicate greater sensitivity to pressure changes.

The differential equation can be solved using numerical methods to predict the yield ( $Y$ ), defined as the fraction of the maximum product released:

$$
Y = \frac{R}{R_m}
$$

By simulating this model, it is possible to estimate the product yield as a function of the number of passes through the homogenizer and optimize operational conditions to maximize process efficiency.
"""

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import plotly.graph_objects as go

#Parameters
R_m = 0.2           #Maximum released product (g/L) 0.2
k = 5.94*10**(-4)   #Kinetic constant (MPa^(-\alpha) 5.94*10**(-4
P = 100             #Pressure (MPa) 100
alpha = 0.22        #\alpha 0.22
N_end = 50000        #Final number of passes 2000

# Initial conditions
R0 = 0.0            # Initial product concentration (g/L)

# Define the system of ODEs:
#This is just a function that returns the ODE's.
# y is the matrix or row where the solution will be built below are the equations needed to
# build the solution
def disruption_ode(t, y, k, P, alpha, R_m):

    R = y[0]                             # The solution will have one row (R)
    dR_dN = k*(P**(alpha))*(R_m - R)     #ODE for the cell disruption using HPH

    return [dR_dN]


def solution_ode(R0, k, P, alpha, R_m, N_end): #This is just a function, no ODE's are being solved

    #Define the function to solve
    fun = lambda t, y: disruption_ode(t, y, k, P, alpha, R_m) # Auxiliar function to reduce the input parameters to use solve_ivp

    #Set initial values for RO - To solve ODE's you always need to give initial parameters and the equations that describe the
    # the variation of that variables.
    y0 = [R0] #é sempre uma lista

    # Passes span
    N_span = (0, N_end) # array with two numbers, 0 and N_end (nº cliclos)
    N_eval = np.linspace(*N_span, 100) #Array with 100 elements to build the ODE's, step by step va t=o y(R), t=1 y(1)i passo a passo:
    #print(N_eval)
    #Solve the differential equation
    solution = solve_ivp(fun, N_span, y0, method='LSODA', t_eval=N_eval, rtol=1e-8, atol=1e-8)

    #Extract the solution
    N = solution.t     #de acordo com o tempo
    R = solution.y[0]  #vem numa lista
    #Y =R/R_m
    Y = [x / R_m for x in R]

    return N, R, Y   #N-tempo, R-libertação, y-yield


# Create figure to represent Y over time
fig1 = go.Figure()

# Use the function solution_monod to get the solution (here you solve the system)
N, R, Y = solution_ode(R0, k, P, alpha, R_m, N_end)

# Add traces for biomass
fig1.add_trace(go.Scatter(x=N, y=Y, mode='lines', name=f'Y'))
#To add other curve simple add a add_trace function below this one

# Update layout for plot
fig1.update_layout(title=f"Disruption Yield (P = {P} MPa)",
                   xaxis_title="Number of passages (N)",
                   yaxis_title="Yield (Y)",
                   width=800)
fig1.show()

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Comparison of disruption yield for different pressure values

R_m = 0.2                        #Maximum released product (g/L)
k = 5.94*10**(-5)                #Kinetic constant (MPa^(-\alpha)
P_values = [10, 20, 35, 50]      #Pressure (MPa)
alpha = 2.9                   #\alpha
N_end = 20                   #Final number of passages

# Initial conditions
R0 = 0.0        # Initial product concentration (g/L)

# Create figure for Concentration in Retentate
fig2 = go.Figure()

# Loop over different mu_max values and plot results
for P in P_values:
    N, R, Y = solution_ode(R0, k, P, alpha, R_m, N_end)

    # Add traces of yield (Y) for each pressure value
    fig2.add_trace(go.Scatter(x=N, y=Y, mode='lines', name=f'P = {P} MPa'))

# Update layout for yield plot
fig2.update_layout(title='Comparison of Yield Disruption for different Pressure Values',
                   xaxis_title="Number of Passages (N)",
                   yaxis_title="Yield (Y)",
                   width=800)

fig2.show()

"""# Diafiltration Process Simulation

This notebook simulates the diafiltration process using an ordinary differential equation (ODE) model to track the concentration of solute in the retentate over time.

## Model Overview

Diafiltration is a separation process in which a solvent (permeate) is continuously removed through a semipermeable membrane while additional solvent is added to the retentate. This process is often used to reduce the concentration of contaminants/impurities while retaining larger molecules like proteins or for buffer exchange processes.

In this model, we track the concentration of a solute in the retentate (`Cc`) over time using the following ODE:


$$
\frac{dCc}{dt} = -\frac{Q_p}{V_0} \cdot (1 - \sigma_a) \cdot Cc
$$

where:
- \( $Q_p$\): Permeate flow rate, representing the rate at which solvent exits the system through the membrane.
- \( $V_0$ \): Initial volume of the retentate.
- \( $\sigma_a$ \): Retention coefficient, which describes the membrane's ability to retain the solute. A value of \( $\sigma_a$ = 1 \) means complete retention of the solute, while \( $\sigma_a$ = 0 \) means complete passage.
- \( $C_c$ \): Solute concentration in the retentate.

## Code Breakdown

1. **Import Libraries**
   - We use `numpy` for numerical calculations, `solve_ivp` from `scipy.integrate` to solve the ODE, and `matplotlib.pyplot` to plot the results.

2. **Define Process Parameters**
    - `Qp`: Permeate flow rate set at 100 L/h.
    - `V0`: Initial volume of the retentate set at 250 L.
    - `sigma_a`: Retention coefficients for the lactase and the impurity. You can set these values as an array or separated variables.
    - `Cc_0`: Initial solute concentration in the retentate for the lactase (2.0 mg/L) and the impurity ( 5.5 mg/L). You can set these values as an array or separated variables.
    - `t_end`: Time range for the simulation set at 10 h.

3. **Define the ODE Function `ode_fcn`** (Previously: `disruption_ode` function)
   - `ode_fcn` calculates the rate of change of solute concentration in the retentate over time.
   - **Inputs**:
     - `t`: Time;
     - `y`: List of dependent variables (here, `y[0]` is `Cc`, the concentration of solute in the retentate);
     - `Qp`, `V0`, `sigma_a`: Parameters for the permeate flow rate, initial retentate volume, and retention coefficient.
   - **Output**:
     - Returns `[dCc_dt]`, the rate of change of solute concentration in the retentate, based on the equation.

4. **Initial Conditions and ODE Solver** (Previously: `solution_ode` function)
   - `y0` is set to the initial concentration `Cc_0`.
   - `solve_ivp` integrates `ode_fcn` over the specified t range (`t_span` and `t_eval` where `t_span` is from 0 to `t_end` and `t_eval` has 1000 elements) and returns the solute concentration in the retentate at each time point.
   - Extract the solutions for t, nD and Cc.

5. **Plotting the Results**
   - The concentration of compounds in the retentate (`Cc`) is plotted against `nD` for the lactase and the impurity to visualize changes during the diafiltration process.

## Questions:

 1) Write a python script, similar to the example of the cell disruption model, to solve the ODE of the diafiltration process;

2)  Plot the concentration of the impurity in the concentrate over nD;

3) Plot the concentration of the lactase in the concentrate over nD;

4) The goal of the diafiltration process is to reduce the concentration of impurity in the retentate from 5.5 mg/L to 1.0 mg/L. What is the minimum diafiltration volumes required to achieve that concentration of impurity?
R:2.43043

5) By achieving 1.0 mg/L of impurity in the retentate, what is the final concentration of lactase?
R:1.5665

6) Finally, how much mass of lactase is lost to the permeate, after achieving 1.0 mg/L of impurity?
R:108.375 mg
2.0-1.5665=0.4335 mg/L * 250.0 (V0) = 108.375 mg
"""

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Process parameters

Qp = 100            # Permeate flow rate (L/h)
V0 = 250.0            # Initial volume of the retentate (L)
sigma_ap = 0.9
sigma_ai = 0.3     # Retention coefficients for the lactase (0.9) and the impurity (0.3)
Cc_0p = 2.0
Cc_0i = 5.5          # Initial solute concentration in the retentate (mg/L)
Cc_04 = 1.0
t_end = 10.0        # Time range for the simulation (hours)

def ode_fcn(t, y, Qp ,V0 ,sigma_a):

    Cc = y[0]                             # The solution will have one row (R)
    dCc_dt = (-(Qp/V0))*(1-sigma_a)*Cc  #ODE for the cell disruption using HPH

    return [dCc_dt]

def solution_ode_fcn(Cc_0, Qp, V0, sigma_a, t_end): #This is just a function, no ODE's are being solved

    #Define the function to solve
    fun = lambda t, y: ode_fcn(t, y, Qp ,V0 ,sigma_a) # Auxiliar function to reduce the input parameters to use solve_ivp

    #Set initial values for X and S - To solve ODE's you always need to give initial parameters and the equations that describe the
    # the variation of that variables.
    y0 = [Cc_0]


    # Passes span
    t_span = (0, t_end) # array with two numbers, 0 and N_end
    t_eval = np.linspace(*t_span, 1000) #Array with 1000 elements to build the ODE's, step by step
    #print(t_eval)
    #Solve the differential equation
    solution = solve_ivp(fun, t_span, y0, method='LSODA', t_eval=t_eval, rtol=1e-8, atol=1e-8)


    #Extract the solution
    t = solution.t
    Cc = solution.y[0]
    #numero de diafiltrações
    nD = [(Qp/V0)*x for x in t]


    return Cc, t, nD


# Create figure to represent Y over time
fig1 = go.Figure()

# Use the function solution_monod to get the solution (here you solve the system)
Cc_p, t, nD = solution_ode_fcn(Cc_0p, Qp, V0, sigma_ap, t_end)
Cc_i, t, nD = solution_ode_fcn(Cc_0i, Qp, V0,sigma_ai, t_end)
# Add traces for biomass
fig1.add_trace(go.Scatter(x=nD, y=Cc_p, mode='lines', name=f'lactose'))
fig1.add_trace(go.Scatter(x=nD, y=Cc_i, mode='lines', name=f'impurity'))
#fig1.add_trace(go.Scatter(x=nD, y=Cc_04, marker='x', s=100, linewidths=2, zorder=5))
#To add other curve simple add a add_trace function below this one

#plt.scatter(Cc_04, color='red', marker='x', s=100, linewidths=2, zorder=5)

# Update layout for plot
fig1.update_layout(title=f"Diafiltration Process)",
                   yaxis_title="Solute concentration (Cc)",
                   xaxis_title="number of Diafiltration (nD)",
                   width=800)
fig1.show()

#Pergunta 4

<>:13: SyntaxWarning: invalid escape sequence '\('
<>:154: SyntaxWarning: invalid escape sequence '\c'
<>:13: SyntaxWarning: invalid escape sequence '\('
<>:154: SyntaxWarning: invalid escape sequence '\c'
/var/folders/gk/fh_m7lqs5ljf6bf6mpt0z0yh0000gn/T/ipykernel_82824/1876192290.py:13: SyntaxWarning: invalid escape sequence '\('
  To model the efficiency of cell disruption in an HPH, a kinetic equation is commonly used. The rate of product release, \( $\frac{dR}{dN}$ \), is described as a function of the applied pressure, the number of passes through the homogenizer, and the maximum achievable product concentration. The equation is as follows:
/var/folders/gk/fh_m7lqs5ljf6bf6mpt0z0yh0000gn/T/ipykernel_82824/1876192290.py:154: SyntaxWarning: invalid escape sequence '\c'
  \frac{dCc}{dt} = -\frac{Q_p}{V_0} \cdot (1 - \sigma_a) \cdot Cc
